**PART 1: Pre Processing + Data Extraction**

In [1]:
'''
1. Pre - Requisite
Install all Library
'''
# 1) Upgrade core tooling inside THIS kernel
%pip install --upgrade pip setuptools wheel --user

# 2) Clean install of audio deps (librosa works with Py3.12 now via numba>=0.61)
%pip install librosa==0.11.0 numba==0.61.2 llvmlite soxr soundfile ffmpeg-python matplotlib tqdm pandas numpy --user

%pip install pandas numpy librosa torchaudio tqdm ffmpeg-python matplotlib soundfile --user
%pip install --upgrade moviepy imageio-ffmpeg torchaudio soundfile numpy tqdm --user

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: librosa==0.11.0 in c:\python312\lib\site-packages (0.11.0)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   -------------------------- ------------- 8.4/12.6 MB 43.5 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 37.7 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.3
    Uninstalling numpy-2.3.3:
      Successfully uninstalled numpy-2.3.3



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.16.1 requires numpy<2.0.0,>=1.26.0; python_version >= "3.12", but you have numpy 2.2.6 which is incompatible.
torchvision 0.19.1+cu121 requires torch==2.4.1+cu121, but you have torch 2.8.0 which is incompatible.


Note: you may need to restart the kernel to use updated packages.


  Using cached numpy-2.3.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.3.3-cp312-cp312-win_amd64.whl (12.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.16.1 requires numpy<2.0.0,>=1.26.0; python_version >= "3.12", but you have numpy 2.3.3 which is incompatible.
torchvision 0.19.1+cu121 requires torch==2.4.1+cu121, but you have torch 2.8.0 which is incompatible.
numba 0.61.2 requires numpy<2.3,>=1.24, but you have numpy 2.3.3 which is incompatible.
scipy 1.13.0 requires numpy<2.3,>=1.22.4, but you have numpy 2.3.3 which is incompatible.


In [4]:
import os, sys, json, random, subprocess, shutil
from pathlib import Path
import numpy as np
import pandas as pd
import librosa, librosa.display
import soundfile as sf
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import Audio, display
from SoccerNet.Downloader import SoccerNetDownloader
import torchaudio

In [5]:
SN_ROOT = Path(r"C:/Users/CYBORG 15/Documents/GitHub/elec5305-project-520038655/SoccerNet")
SN_ROOT.mkdir(parents=True, exist_ok=True)

SOCCERNET_NDA = 's0cc3rn3t'

# Workspace
WORK_DIR  = Path("./sn_audio_work"); WORK_DIR.mkdir(parents=True, exist_ok=True)
AUDIO_DIR = WORK_DIR / "full_match_audio_wav"; AUDIO_DIR.mkdir(parents=True, exist_ok=True)
CLIPS_DIR = WORK_DIR / "clips";               CLIPS_DIR.mkdir(parents=True, exist_ok=True)
FEAT_DIR  = WORK_DIR / "mel64";               FEAT_DIR.mkdir(parents=True, exist_ok=True)

# Audio + reproducibility
TARGET_SR    = 16000
WINDOWS      = [15, 30]   # 15s/30s event windows
RANDOM_SEED  = 1337
random.seed(RANDOM_SEED)
torchaudio.set_audio_backend("soundfile")  # stable on Windows

print("SN_ROOT:", SN_ROOT)
print("WORK_DIR:", WORK_DIR.resolve())


SN_ROOT: C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet
WORK_DIR: C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\sn_audio_work


C:\Users\CYBORG 15\AppData\Local\Temp\ipykernel_26140\959740583.py:17: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.set_audio_backend("soundfile")  # stable on Windows
C:\Users\CYBORG 15\AppData\Roaming\Python\Python312\site-packages\torchaudio\_internal\module_utils.py:71: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  return func(*args, **kwargs)


In [6]:
from pathlib import Path
from SoccerNet.Downloader import SoccerNetDownloader
import json

def find_label_files(root: Path):
    files = list(root.rglob("Labels-v2.json"))
    if not files:
        files = list(root.rglob("Labels.json"))
    print(f"Found label files: {len(files)}")
    return files

def _parse_time_mmss(s: str):
    parts = s.split(":")
    try:
        if len(parts)==3:
            hh,mm,ss = parts; return int(hh)*3600 + int(mm)*60 + float(ss)
        if len(parts)==2:
            mm,ss = parts;    return int(mm)*60 + float(ss)
    except Exception:
        return None
    return None

def load_labels_with_half(labels_path: Path):
    """Return list of dicts: {'time_s': float, 'label': str, 'half': 0|1|2}"""
    with open(labels_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    ann = data.get("annotations") or data.get("events") or data.get("labels") or []
    out=[]
    for ev in ann:
        lab  = ev.get("label") or ev.get("event") or ev.get("type")
        if not lab:
            continue
        half = ev.get("half")
        if half not in (1,2):
            gt = ev.get("gameTime")
            if isinstance(gt, str) and " - " in gt:
                try: half = int(gt.split(" - ")[0].strip())
                except: half = None
        pos = ev.get("position")
        t = None
        if isinstance(pos,(int,float)): t = float(pos)
        elif isinstance(pos,str):       t = _parse_time_mmss(pos)
        if t is None:
            gt = ev.get("gameTime")
            if isinstance(gt, str) and " - " in gt:
                t = _parse_time_mmss(gt.split(" - ")[1].strip())
        if t is None:
            continue
        out.append({"time_s": float(t), "label": lab, "half": (half if half in (1,2) else 0)})
    return out

In [7]:
def find_half_videos(match_dir: Path):
    """Return {'1': Path|None, '2': Path|None, 'any': Path|None}"""
    half_map = {1: None, 2: None}
    any_vid = None
    preferred = [
        ("1_720p.mkv", 1), ("2_720p.mkv", 2),
        ("1_224p.mkv", 1), ("2_224p.mkv", 2),
        ("1.mkv", 1),     ("2.mkv", 2),
        ("1.mp4", 1),     ("2.mp4", 2),
    ]
    for name, half in preferred:
        p = match_dir / name
        if p.exists():
            half_map[half] = p
            if any_vid is None: any_vid = p
    if any_vid is None:
        for ext in (".mkv",".mp4",".m4v",".mov"):
            found = list(match_dir.glob(f"*{ext}"))
            if found:
                any_vid = found[0]; break
    return {"1": half_map[1], "2": half_map[2], "any": any_vid}


In [8]:
rows=[]
for lf in tqdm(find_label_files(SN_ROOT), desc="Index labels"):
    mdir  = lf.parent
    split = mdir.parent.name
    evs   = load_labels_with_half(lf)
    if not evs:
        continue
    vids  = find_half_videos(mdir)  # may be all None if videos not yet downloaded
    for e in evs:
        # choose video by half when available, else fallback to any
        video = vids[str(e["half"])] if e["half"] in (1,2) else vids["any"]
        rows.append({
            "split": split,
            "match_dir": str(mdir),
            "labels_json": str(lf),
            "event_time_s": e["time_s"],
            "event_label": e["label"],
            "half": int(e["half"]),
            "video_path": (str(video) if video is not None else None)
        })

df_events = pd.DataFrame(rows).drop_duplicates(
    subset=["match_dir","event_time_s","event_label","half"]
).reset_index(drop=True)

print("Indexed events:", len(df_events))
print("Have video_path for:", df_events["video_path"].notna().sum(), "events")
ALL_LABELS = sorted(df_events["event_label"].unique())
print("Classes:", ALL_LABELS, "| total:", len(ALL_LABELS))


Found label files: 500


Index labels: 100%|██████████| 500/500 [00:01<00:00, 468.28it/s]


Indexed events: 110304
Have video_path for: 195 events
Classes: ['Ball out of play', 'Clearance', 'Corner', 'Direct free-kick', 'Foul', 'Goal', 'Indirect free-kick', 'Kick-off', 'Offside', 'Penalty', 'Red card', 'Shots off target', 'Shots on target', 'Substitution', 'Throw-in', 'Yellow card', 'Yellow->red card'] | total: 17


In [21]:
from SoccerNet.Downloader import SoccerNetDownloader
from pathlib import Path

# Create an instance, pointing to your dataset root
mySoccerNetDownloader = SoccerNetDownloader(LocalDirectory=str(SN_ROOT))
mySoccerNetDownloader.password = SOCCERNET_NDA  # 's0cc3rn3t'

print("SoccerNet root:", SN_ROOT)

# ✅ Download labels (small files, safe)
mySoccerNetDownloader.downloadDataTask(task="labels", split=["train","valid","test","challenge"])

# ✅ Download 224p halves, but exclude spain_laliga
splits = ["train","valid","test","challenge"]
for s in splits:
    split_dir = Path(SN_ROOT) / s
    if not split_dir.exists():
        continue
    for comp_dir in split_dir.iterdir():
        if not comp_dir.is_dir():
            continue
        if "spain_laliga" in comp_dir.name:
            print(f"⏭️ Skipping competition: {comp_dir}")
            continue
        print(f"⬇️ Downloading videos for competition: {comp_dir}")
        mySoccerNetDownloader.downloadGames(
            files=["1_224p.mkv", "2_224p.mkv"],
            split=[s]
        )
# Create an instance, pointing to your dataset root
mySoccerNetDownloader = SoccerNetDownloader(LocalDirectory=str(SN_ROOT))
mySoccerNetDownloader.password = SOCCERNET_NDA  # 's0cc3rn3t'

print("SoccerNet root:", SN_ROOT)

# ✅ Ensure labels are present (small JSON files, will skip if already exist)
mySoccerNetDownloader.downloadDataTask(task="labels", split=["train","valid","test","challenge"])

# Instead: scan and list what’s already downloaded
video_exts = {".mkv", ".mp4", ".m4v", ".mov"}
downloaded_videos = []
for f in SN_ROOT.rglob("*"):
    if f.suffix in video_exts and f.name.startswith(("1_", "2_")):
        downloaded_videos.append(f)

print(f"[Scan] Total video halves found: {len(downloaded_videos)}")
if downloaded_videos:
    # Show just the first 10 to avoid huge spam
    for v in sorted(downloaded_videos)[:10]:
        print(" -", v.relative_to(SN_ROOT))
    if len(downloaded_videos) > 10:
        print(f"... and {len(downloaded_videos)-10} more")
else:
    print("⚠️ No video halves found. Check your dataset folder.")

SoccerNet root: C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet
ERROR Unknown task: labels
SoccerNet root: C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet
ERROR Unknown task: labels
[Scan] Total video halves found: 470
 - england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\1_224p.mkv
 - england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\2_224p.mkv
 - england_epl\2014-2015\2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal\1_224p.mkv
 - england_epl\2014-2015\2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal\2_224p.mkv
 - england_epl\2014-2015\2015-02-21 - 18-00 Swansea 2 - 1 Manchester United\1_224p.mkv
 - england_epl\2014-2015\2015-02-21 - 18-00 Swansea 2 - 1 Manchester United\2_224p.mkv
 - england_epl\2014-2015\2015-02-22 - 19-15 Southampton 0 - 2 Liverpool\1_224p.mkv
 - england_epl\2014-2015\2015-02-22 - 19-15 Southampton 0 - 2 Liverpool\2_224p.mkv
 - england_epl\2014-2015\2015-04-11 - 19-30 Burnley 0 - 1 Arsenal\1_7

In [14]:
from pathlib import Path
import re, shutil
import pandas as pd
from SoccerNet.Downloader import SoccerNetDownloader

# --- config ---
NEEDED_FILES = ["1_224p.mkv", "2_224p.mkv"]
DROP_COMP     = "spain_laliga"
CANON_COMPS   = [
    "england_epl",
    "europe_uefa-champions-league",
    "france_ligue-1",
    "germany_bundesliga",
    "italy_serie-a",
]

# --- helper: parse competition/season from your on-disk layout ---
SEASON_RE = re.compile(r"^\d{4}-\d{4}$")
def find_label_game_dirs(root: Path):
    return [p.parent for p in root.rglob("Labels*.json")]

def parse_comp_season_from_game_dir(game_dir: Path, root: Path):
    parts = game_dir.relative_to(root).parts
    comp = None; season = None
    for i, p in enumerate(parts):
        if p in (set(CANON_COMPS) | {DROP_COMP}):
            comp = p
            if i+1 < len(parts) and SEASON_RE.match(parts[i+1]):
                season = parts[i+1]
            break
    if comp is None and len(parts) >= 1:
        comp = parts[0]
    if season is None:
        for p in parts:
            if SEASON_RE.match(p): season = p; break
    return comp, season

def list_kept_game_dirs(root: Path, keep_comps: list[str]):
    out = []
    for gd in find_label_game_dirs(root):
        comp, _ = parse_comp_season_from_game_dir(gd, root)
        if comp in keep_comps:
            out.append(gd)
    return out

def check_missing_halves(game_dirs: list[Path], need_files=NEEDED_FILES):
    missing = []
    have = 0
    for g in game_dirs:
        miss = [f for f in need_files if not (g / f).exists()]
        if miss: missing.append((g, miss))
        else:    have += 1
    return have, missing

# --- 0) Make a downloader instance (for labels if ever needed) ---
mySoccerNetDownloader = SoccerNetDownloader(LocalDirectory=str(SN_ROOT))
mySoccerNetDownloader.password = SOCCERNET_NDA

print("SoccerNet root:", SN_ROOT)

# --- 1) Ensure labels exist (fast & small) ---
# Safe to call repeatedly; it skips existing files.
mySoccerNetDownloader.downloadDataTask(task="labels", split=["train","valid","test","challenge"])

# --- 2) Inventory competitions present from labels ---
label_games = find_label_game_dirs(SN_ROOT)
print(f"[Labels] Games with labels found: {len(label_games)}")

inventory = {}
for gd in label_games:
    comp, season = parse_comp_season_from_game_dir(gd, SN_ROOT)
    inventory.setdefault(comp, set())
    if season: inventory[comp].add(season)

# Warn if LaLiga exists; delete or ignore as you wish (here: just ignore printing)
if DROP_COMP in inventory and len(inventory[DROP_COMP]) > 0:
    print(f"⚠️ Found '{DROP_COMP}' labels on disk. They will be excluded from processing.")
else:
    print("✅ 'spain_laliga' not present — nothing to exclude.")

# --- 3) Build kept list (exclude LaLiga; keep canonical comps that exist) ---
KEEP_COMPETITIONS = [c for c in CANON_COMPS if c in inventory]
print("Keeping competitions:", KEEP_COMPETITIONS)

# --- 4) Filter to kept games and check which are complete (both halves) ---
keep_game_dirs = list_kept_game_dirs(SN_ROOT, KEEP_COMPETITIONS)
have_count, missing_list = check_missing_halves(keep_game_dirs, NEEDED_FILES)

print(f"[Scan] Kept games (labels): {len(keep_game_dirs)}")
print(f"[Scan] Complete (both halves present): {have_count}")
print(f"[Scan] Incomplete/missing: {len(missing_list)}")

# --- 5) Use ONLY the complete games; no further downloads requested ---
complete_games = [g for g in keep_game_dirs if all((g/f).exists() for f in NEEDED_FILES)]

# Small assertion/print
if len(complete_games) == 0:
    print("❌ No complete games found. If this is unexpected, double-check paths or consider downloading missing halves.")
else:
    print(f"✅ All data ready: using {len(complete_games)} complete games (excluding '{DROP_COMP}').")

# --- 6) Save list of complete game directories for downstream preprocessing ---
complete_df = pd.DataFrame({"game_dir": [str(g) for g in complete_games]})
COMPLETE_LIST_CSV = (WORK_DIR / "complete_games_224p.csv")
complete_df.to_csv(COMPLETE_LIST_CSV, index=False)
print("Saved list of complete games to:", COMPLETE_LIST_CSV)


SoccerNet root: C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet
ERROR Unknown task: labels
[Labels] Games with labels found: 383
✅ 'spain_laliga' not present — nothing to exclude.
Keeping competitions: ['england_epl', 'europe_uefa-champions-league', 'france_ligue-1', 'germany_bundesliga', 'italy_serie-a']
[Scan] Kept games (labels): 383
[Scan] Complete (both halves present): 234
[Scan] Incomplete/missing: 149
✅ All data ready: using 234 complete games (excluding 'spain_laliga').
Saved list of complete games to: sn_audio_work\complete_games_224p.csv


In [11]:
from pathlib import Path

def find_label_game_dirs(root: Path):
    """Return list of game dirs that contain Labels*.json."""
    games = []
    for p in root.rglob("Labels*.json"):
        # game dir = parent of label file
        games.append(p.parent)
    return games

def list_competitions_from_labels(root: Path):
    """Return sorted list of (split, competition) tuples present on disk."""
    comps = set()
    for game_dir in find_label_game_dirs(root):
        # Expect: root / split / competition / season / game
        parts = game_dir.relative_to(root).parts
        if len(parts) >= 2:
            split, competition = parts[0], parts[1]
            comps.add((split, competition))
    return sorted(comps)

label_games = find_label_game_dirs(SN_ROOT)
print(f"Games with labels found: {len(label_games)}")
detected = list_competitions_from_labels(SN_ROOT)
print("Detected (split, competition) present from labels:")
for s,c in detected[:15]:
    print(" -", s, "|", c)
print("Total competitions detected:", len({c for _,c in detected}))


Games with labels found: 500
Detected (split, competition) present from labels:
 - england_epl | 2014-2015
 - england_epl | 2015-2016
 - england_epl | 2016-2017
 - europe_uefa-champions-league | 2014-2015
 - europe_uefa-champions-league | 2015-2016
 - europe_uefa-champions-league | 2016-2017
 - france_ligue-1 | 2014-2015
 - france_ligue-1 | 2015-2016
 - france_ligue-1 | 2016-2017
 - germany_bundesliga | 2014-2015
 - germany_bundesliga | 2015-2016
 - germany_bundesliga | 2016-2017
 - italy_serie-a | 2014-2015
 - italy_serie-a | 2015-2016
 - italy_serie-a | 2016-2017
Total competitions detected: 3


In [20]:
from pathlib import Path
import re

KNOWN_COMPS = {
    "england_epl",
    "europe_uefa-champions-league",
    "france_ligue-1",
    "germany_bundesliga",
    "italy_serie-a",
    "spain_laliga",
}

SEASON_RE = re.compile(r"^\d{4}-\d{4}$")  # e.g., 2014-2015

def parse_comp_season_from_game_dir(game_dir: Path, root: Path):
    """
    Expect: root / <competition> / <season> / <game> / Labels*.json
    Returns (competition, season) or (None, None) if not found.
    """
    parts = game_dir.relative_to(root).parts  # e.g., ('italy_serie-a','2015-2016','game_id')
    comp = None
    season = None
    for i, p in enumerate(parts):
        if p in KNOWN_COMPS:
            comp = p
            # try pick immediate next part as season if it looks like YYYY-YYYY
            if i+1 < len(parts) and SEASON_RE.match(parts[i+1]):
                season = parts[i+1]
            break
    # Fallbacks
    if comp is None and len(parts) >= 1:
        comp = parts[0]
    if season is None:
        for p in parts:
            if SEASON_RE.match(p):
                season = p
                break
    return comp, season

def find_label_game_dirs(root: Path):
    """Return all game dirs that contain Labels*.json."""
    games = []
    for p in root.rglob("Labels*.json"):
        games.append(p.parent)
    return games

# Inspect what you actually have
label_games = find_label_game_dirs(SN_ROOT)
print("Games with labels found:", len(label_games))

# Build (competition, season) inventory
inventory = {}
for gd in label_games:
    comp, season = parse_comp_season_from_game_dir(gd, SN_ROOT)
    if comp is None: 
        comp = "(unknown)"
    seasons = inventory.setdefault(comp, set())
    if season:
        seasons.add(season)

print("Competitions detected:")
for c in sorted(inventory.keys()):
    print(f" - {c}: {sorted(inventory[c])[:5]}{' ...' if len(inventory[c])>5 else ''}")

# Canonical order that matches your naming
CANON_ORDER = [
    "england_epl",
    "europe_uefa-champions-league",
    "france_ligue-1",
    "germany_bundesliga",
    "italy_serie-a",
    "spain_laliga",
]

present_comps = [c for c in CANON_ORDER if c in inventory]
print("Present comps in canonical order:", present_comps)

KEEP_UP_TO = "italy_serie-a"
DROP_COMP  = "spain_laliga"

if KEEP_UP_TO in present_comps:
    keep_idx = present_comps.index(KEEP_UP_TO)
    KEEP_COMPETITIONS = present_comps[:keep_idx+1]
else:
    # Fallback: keep all present except Spain
    KEEP_COMPETITIONS = [c for c in present_comps if c != DROP_COMP]

print("KEEP:", KEEP_COMPETITIONS)

NEEDED_FILES = ["1_224p.mkv", "2_224p.mkv"]  # change to 720p if you prefer

def list_kept_game_dirs(root: Path, keep_comps: list[str]):
    """All game dirs under kept competitions (based on where labels are)."""
    out = []
    for gd in find_label_game_dirs(root):
        comp, season = parse_comp_season_from_game_dir(gd, root)
        if comp in keep_comps:
            out.append(gd)
    return out

def check_missing_halves(game_dirs: list[Path], need_files=NEEDED_FILES):
    missing = []
    have = 0
    for g in game_dirs:
        miss = [f for f in need_files if not (g / f).exists()]
        if miss:
            missing.append((g, miss))
        else:
            have += 1
    return have, missing

keep_game_dirs = list_kept_game_dirs(SN_ROOT, KEEP_COMPETITIONS)
have_count, missing_list = check_missing_halves(keep_game_dirs, NEEDED_FILES)

print(f"[Scan] Kept games (with labels): {len(keep_game_dirs)}")
print(f"[Scan] Complete (both halves present): {have_count}")


Games with labels found: 383
Competitions detected:
 - england_epl: ['2014-2015', '2015-2016', '2016-2017']
 - europe_uefa-champions-league: ['2014-2015', '2015-2016', '2016-2017']
 - france_ligue-1: ['2014-2015', '2015-2016', '2016-2017']
 - germany_bundesliga: ['2014-2015', '2015-2016', '2016-2017']
 - italy_serie-a: ['2014-2015', '2015-2016', '2016-2017']
Present comps in canonical order: ['england_epl', 'europe_uefa-champions-league', 'france_ligue-1', 'germany_bundesliga', 'italy_serie-a']
KEEP: ['england_epl', 'europe_uefa-champions-league', 'france_ligue-1', 'germany_bundesliga', 'italy_serie-a']
[Scan] Kept games (with labels): 383
[Scan] Complete (both halves present): 234
